# Getting Started in AMICI

This notebook is a brief tutorial for new users that explains the first steps necessary for model simulation in AMICI, including pointers to documentation and more advanced notebooks.

## Model Compilation
Before simulations can be run, the model must be imported and compiled. In this process, AMICI performs all symbolic manipulations that later enable scalable simulations and efficient sensitivity computation. The first step towards model compilation is the creation of an [SbmlImporter](https://amici.readthedocs.io/en/latest/generated/amici.sbml_import.SbmlImporter.html) instance, which requires an SBML Document that specifies the model using the [Systems Biology Markup Language (SBML)](https://sbml.org/).

For the purpose of this tutorial, we will use `model_steadystate_scaled.xml`, which is contained in the same directory as this notebook.

In [ ]:
import amici
import numpy as np

sbml_importer = amici.SbmlImporter("model_steadystate_scaled.xml")

Next, we will compile the model as python extension using the [amici.SBMLImporter.sbml2amici](https://amici.readthedocs.io/en/latest/generated/amici.sbml_import.SbmlImporter.html#amici.sbml_import.SbmlImporter.sbml2amici) method. The first two arguments of this method are the name of the model, which will also be the name of the generated python module, and the model directory, which defines the directory in which the model module will be placed. Compilation will take a couple of seconds.

In [ ]:
model_name = "model_steadystate"
model_dir = "model_dir"
sbml_importer.sbml2amici(model_name, model_dir)

## Loading the model module
To run simulations, we need to instantiate [amici.Model](https://amici.readthedocs.io/en/latest/generated/amici.amici.Model.html) and [amici.Solver](https://amici.readthedocs.io/en/latest/generated/amici.amici.Solver.html) instances. As simulations require instances matching the imported model, they have to be imported from the generated model module.

In [ ]:
# load the model module
model_module = amici.import_model_module(model_name, model_dir)
# instantiate model
model = model_module.get_model()
# instantiate solver
solver = model.create_solver()

The model allows the user to manipulate model related properties of simulations. This includes the values of model parameters that can be set by using [Model.set_free_parameter_by_name](https://amici.readthedocs.io/en/latest/generated/amici.sim.sundials.html#amici.sim.sundials.Model.set_free_parameter_by_name). Here, we set the model parameter `p1` to a value of `1e-3`.

In [ ]:
model.set_free_parameter_by_name("p1", 1e-3)

In contrast, the solver instance allows the specification of simulation related properties. This includes setting options for the SUNDIALS solver such as absolute tolerances via [Solver.set_absolute_tolerance](https://amici.readthedocs.io/en/latest/generated/amici.sim.sundials.html#amici.sim.sundials.Solver.set_absolute_tolerance). Here we set the absolute integration tolerances to `1e-10`.

In [ ]:
solver.set_absolute_tolerance(1e-10)

## Running Model Simulations

Model simulations can be executed using the [Model.simulate](https://amici.readthedocs.io/en/latest/generated/amici.sim.sundials.html#amici.sim.sundials.Model.simulate) method (or, alternatively, [amici.run_simulation](https://amici.readthedocs.io/en/latest/generated/amici.sim.sundials.html#amici.sim.sundials.run_simulation)). By default, the model does not contain any output timepoints for which the model is to be simulated. Here we define a simulation timecourse with two output timepoints at `0` and `1` and then run the simulation.

In [ ]:
# set timepoints
model.set_timepoints(np.linspace(0, 10, 101))
rdata = model.simulate(solver=solver)

Simulation results are returned as [ReturnData](https://amici.readthedocs.io/en/latest/generated/amici.sim.sundials.html#amici.sim.sundials.ReturnData) instance. The simulated SBML species are stored as `x` attribute, where rows correspond to the different timepoints and columns correspond to different species.

In [ ]:
rdata.x

All results attributes are always ordered according to the model. For species, this means that the columns of `rdata.x` match the ordering of species in the model, which can be accessed as [Model.get_state_names](https://amici.readthedocs.io/en/latest/generated/amici.sim.sundials.html#amici.sim.sundials.Model.get_state_names) or as `ReturnData.state_names`:

In [ ]:
model.get_state_names()

For convenience, most results stored in `ReturnData` can also be retrieved as [xarray.DataArray](https://docs.xarray.dev/en/stable/index.html) objects that already include the respective row and column names. This can be accessed via the `xr` attribute of `ReturnData`. Here, we access the model state `x` as `DataArray` object to convert it to a `pandas.DataFrame`:

In [ ]:
rdata.xr.x.to_pandas()

In [ ]:
from amici.sim.sundials.plotting import *

plot_state_trajectories(rdata)

This notebook only explains the basics of AMICI simulations. In general, AMICI simulations are highly customizable and can also be used to simulate sensitivities. The [extended example](https://amici.readthedocs.io/en/latest/examples/getting_started_extended/GettingStartedExtended.html) notebook gives more detail about the model employed here and goes into the basics of sensitivity analysis. The [ExampleEquilibrationLogic](https://amici.readthedocs.io/en/latest/examples/example_steady_states/ExampleEquilibrationLogic.html) notebook, builds on this by using a modified version of this model to give detailed insights into the methods and options to compute steady states before and after simulations, as well as respective sensitivities. The [ExampleExperimentalConditions example](https://amici.readthedocs.io/en/latest/examples/example_presimulation/ExampleExperimentalConditions.html) notebook, goes into the details of how even more complex experimental setups, such as addition of drugs at predefined timepoints, can be simulated in AMICI. Finally, the [PEtab](https://amici.readthedocs.io/en/latest/examples/example_petab/petab.html) notebook explains how standardized definitions of experimental data and conditions in the [PEtab](https://github.com/PEtab-dev/PEtab) format can be imported in AMICI.